<a href="https://www.kaggle.com/code/soumikniloy/pet-s-facial-expression-image-dataset?scriptVersionId=140381231" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# Set the paths to your dataset folders
train_data_dir = '/kaggle/input/pets-facial-expression-dataset/Master Folder/train'
valid_data_dir = '/kaggle/input/pets-facial-expression-dataset/Master Folder/test'

In [3]:
# Set hyperparameters
batch_size = 32
epochs = 10
input_shape = (128, 128, 3)  

In [4]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1000 images belonging to 4 classes.
Found 38 images belonging to 4 classes.


In [5]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')  
])

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size
)

Epoch 1/10
31/31 [==============================] - 21s 610ms/step - loss: 1.4810 - accuracy: 0.2479 - val_loss: 1.3985 - val_accuracy: 0.3438
Epoch 2/10
31/31 [==============================] - 19s 619ms/step - loss: 1.3766 - accuracy: 0.2614 - val_loss: 1.4816 - val_accuracy: 0.2500
Epoch 3/10
31/31 [==============================] - 22s 704ms/step - loss: 1.3754 - accuracy: 0.2843 - val_loss: 1.4201 - val_accuracy: 0.3438
Epoch 4/10
31/31 [==============================] - 19s 616ms/step - loss: 1.3614 - accuracy: 0.3006 - val_loss: 1.4770 - val_accuracy: 0.3750
Epoch 5/10
31/31 [==============================] - 20s 639ms/step - loss: 1.3414 - accuracy: 0.3286 - val_loss: 1.6625 - val_accuracy: 0.2500
Epoch 6/10
31/31 [==============================] - 20s 633ms/step - loss: 1.3317 - accuracy: 0.3440 - val_loss: 1.7366 - val_accuracy: 0.3125
Epoch 7/10
31/31 [==============================] - 19s 618ms/step - loss: 1.3368 - accuracy: 0.3378 - val_loss: 1.5565 - val_accuracy: 0.3125

In [8]:
# Evaluate the model
test_data_dir = '/kaggle/input/pets-facial-expression-dataset/Master Folder/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_generator)

print("Test accuracy:", test_acc)

Found 38 images belonging to 4 classes.
2/2 [==============================] - 0s 36ms/step - loss: 1.7722 - accuracy: 0.2895
Test accuracy: 0.28947368264198303


In [9]:
model.save('path_to_saved_model')

In [10]:
# Load the pre-trained model
model = tf.keras.models.load_model('path_to_saved_model') 

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [12]:
# Load and preprocess the input image
input_image_path = '/kaggle/input/pets-facial-expression-dataset/Master Folder/test/Angry/01.jpg'  
input_image = image.load_img(input_image_path, target_size=input_shape[:2])
input_image_array = image.img_to_array(input_image)
input_image_array = np.expand_dims(input_image_array, axis=0)
input_image_array /= 255.0  # Normalize pixel values

In [13]:
# Use the model to predict the class of the input image
predictions = model.predict(input_image_array)

# Get the class label with the highest probability
predicted_class = np.argmax(predictions)

1/1 [==============================] - 0s 155ms/step


In [14]:
# Display the prediction result
class_names = train_generator.class_indices  # Get the class names from the training generator
for class_label, class_index in class_names.items():
    if class_index == predicted_class:
        predicted_class_name = class_label

print("Predicted class:", predicted_class_name)
print("Predictions:", predictions)

Predicted class: happy
Predictions: [[0.23752266 0.18787132 0.16303004 0.41157597]]
